<a href="https://colab.research.google.com/github/yong-again/seq2seq_lstm/blob/lyj/seq2seq_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('안녕하세요 2021년 12월 29일 입니다.')

In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma
import warnings

import os
import shutil
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_excel('./1_구어체(1).xlsx', names=["SID", "src", "tar"])
df = df[:10000].copy()

In [ ]:
del df['SID']

In [ ]:
len(df)

10000

In [ ]:
df.head(10)

,src,tar
0,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,씨티은행에서 일하세요?,Do you work at a City bank?
2,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...
5,F/W 겐조타이거 키즈와 그리고 이번에 주문한 키즈 중 부족한 수량에 대한 환불입니다.,"18fw Kenzo Tiger Kids, and refund for lacking ..."
6,강아지들과 내 사진을 보낼게.,And I'll send you a picture of me and my dogs.
7,그 수익금 중 일부를 위안부 할머니들을 위해 쓰고 그들을 위해 여러 가지 캠페인을 ...,Part of profits are used for the comfort women...
8,그들은 내가 잘하는 것을 바탕으로 별명을 사용하고 있기 때문에 나는 사람들이 치타라...,I feel happy when people call me cheetah becau...
9,그러므로 실제로 컴퓨터 프로그램을 만든 사람이 프로그램에 대한 저작자가 돼요.,"So, a person who made a computer program actua..."


In [ ]:
df.tar = df.tar.apply(lambda x: '\t ' + x + ' \n')
df.sample(10)

,src,tar
18,"얼마 전 가진 A사와의 회의 결과, 본계약의 계약 시점은 약간 지연된다고 합니다.",\t As a result of the meeting with A company t...
4657,A 대학교는 항공과 해양스포츠 분야의 수준 높은 시설과 교육을 겸비한 학교입니다.,\t A university is combined with high-class fa...
8393,가장 인기 있는 호텔은 시내 접근성이 좋고 저렴한 호텔입니다.,\t The most popular hotel has good accessibili...
3256,2번 사진은 내가 당신에게 준 표본입니다.,\t Picture 2 is the sample that I gave to you. \n
1278,캐시는 이미 결혼한 사람이에요.,\t Kathy is already married. \n
6878,Smith씨 댁으로 가게 된 교환학생 Ann입니다.,\t My name is Ann who is an exchange student t...
2535,1월 12-13일이라는 마감일의 기준이 어떻게 산정된 건가?,\t How did you come up with January 12th-13th ...
8891,각 이슈에 대해 네가 명확히 하고자 하는 것들은 광진을 통해 충분히 전달될 것이에요.,\t The things you would like to clear in each ...
9349,감독으로써 화려한 경력을 가진 김성근 감독의 영입이 있었으니까 이번 시즌 성적이 기...,"\t As he has such a wonderful career, the futu..."
3775,3일 동안 일만 하는 건 매우 힘들어요.,\t It's really hard to work for 3 days. \n


In [ ]:
# 문자 집합 구축
src_vocab = set()

for data in df.src:
    for char in data:
        src_vocab.add(char)

tar_vocab = set()
for data in df.tar:
    for char in data:
        tar_vocab.add(char)

In [ ]:
# 문자 개수
src_vocab_size = len(src_vocab) 
tar_vocab_size = len(tar_vocab) 

print(src_vocab_size, tar_vocab_size)

1390 115


In [ ]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[100:150])
print(tar_vocab[45:75])

['가', '각', '간', '갇', '갈', '갉', '감', '갑', '값', '갓', '갔', '강', '갖', '같', '갚', '개', '객', '갤', '갯', '갱', '갸', '걀', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '겉', '게', '겐', '겟', '겠', '겨', '격', '겪', '견', '결', '겸', '겹', '겼', '경', '곁', '계', '고', '곡', '곤', '곧']
['M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']


In [ ]:
# 문자 집합에 인덱스 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, '<': 25, '=': 26, '>': 27, '?': 28, 'A': 29, 'B': 30, 'C': 31, 'D': 32, 'E': 33, 'F': 34, 'G': 35, 'H': 36, 'I': 37, 'J': 38, 'K': 39, 'L': 40, 'M': 41, 'N': 42, 'O': 43, 'P': 44, 'Q': 45, 'R': 46, 'S': 47, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'X': 52, 'Y': 53, 'Z': 54, '_': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81, '~': 82, '²': 83, '·': 84, '˚': 85, 'Ω': 86, '‘': 87, '“': 88, '․': 89, '…': 90, '℃': 91, 'Ⅳ': 92, '∼': 93, '「': 94, '」': 95, '㎛': 96, '㎡': 97, '情': 98, '社': 99, '美': 100, '가': 101, '각': 102, '간': 103, '갇': 104, '갈': 105, '갉': 106, '감': 107, '갑': 108, '값': 109, '갓': 110, '갔': 11

In [ ]:
# src 정수 인코딩
encoder_input = []
for data in df.src:
    encoded_line = []
    for char in data:
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)
    
print(encoder_input[:5])

[[7, 30, 64, 57, 67, 60, 1, 31, 70, 67, 70, 73, 64, 69, 62, 7, 953, 1, 748, 144, 958, 1, 842, 530, 342, 928, 1, 959, 862, 195, 529, 1, 1101, 1325, 1, 1309, 1, 775, 1, 968, 328, 1, 1149, 476, 541, 1, 860, 966, 334, 342, 12], [836, 1243, 953, 1320, 886, 739, 1, 962, 1306, 749, 922, 28], [1287, 534, 1222, 958, 1, 645, 792, 1238, 752, 476, 328, 1, 1314, 920, 886, 739, 1, 966, 761, 593, 544, 952, 501, 1, 18, 1074, 1, 914, 1252, 954, 1, 195, 502, 1306, 898, 342, 12], [15, 15, 981, 886, 739, 328, 1, 901, 775, 338, 959, 1, 959, 639, 888, 1, 258, 717, 501, 529, 1, 590, 365, 886, 739, 1, 671, 476, 272, 870, 1, 1015, 953, 1, 972, 1, 101, 928, 370, 739, 1, 720, 534, 758, 796, 334, 342, 12], [20, 12, 19, 10, 1, 21, 10, 1, 22, 1, 717, 959, 1027, 101, 1, 580, 1, 116, 258, 1, 361, 1, 983, 966, 147, 1, 388, 1034, 1, 996, 132, 1, 847, 492, 1014, 799, 577, 1, 107, 717, 1306, 135, 796, 334, 342, 12]]


In [ ]:
# tar 정수 인코딩
decoder_input = []
for data in df.tar:
    encoded_line = []
    for char in data:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)
print(decoder_input[:5])

[[1, 3, 35, 72, 65, 75, 68, 3, 36, 78, 75, 78, 81, 72, 77, 70, 9, 3, 72, 82, 3, 64, 3, 66, 78, 75, 78, 81, 72, 77, 70, 3, 64, 79, 79, 75, 72, 66, 64, 83, 72, 78, 77, 3, 83, 71, 64, 83, 3, 64, 75, 75, 78, 86, 82, 3, 88, 78, 84, 3, 83, 78, 3, 68, 87, 79, 68, 81, 72, 68, 77, 66, 68, 3, 65, 68, 64, 84, 83, 72, 69, 84, 75, 3, 82, 83, 78, 81, 72, 68, 82, 3, 72, 77, 3, 83, 71, 68, 3, 35, 72, 65, 75, 68, 16, 3, 2], [1, 3, 37, 78, 3, 88, 78, 84, 3, 86, 78, 81, 74, 3, 64, 83, 3, 64, 3, 36, 72, 83, 88, 3, 65, 64, 77, 74, 33, 3, 2], [1, 3, 49, 54, 51, 42, 53, 48, 9, 82, 3, 65, 68, 82, 83, 82, 68, 75, 75, 68, 81, 14, 3, 86, 71, 72, 66, 71, 3, 81, 68, 66, 78, 81, 67, 68, 67, 3, 22, 83, 71, 3, 81, 78, 84, 70, 71, 3, 15, 66, 84, 83, 82, 3, 65, 88, 3, 86, 78, 81, 67, 82, 3, 78, 69, 3, 76, 78, 84, 83, 71, 3, 69, 81, 78, 76, 3, 64, 65, 81, 78, 64, 67, 16, 3, 2], [1, 3, 42, 77, 3, 36, 71, 64, 79, 83, 68, 81, 3, 19, 19, 3, 43, 68, 82, 84, 82, 3, 66, 64, 75, 75, 68, 67, 3, 45, 64, 89, 64, 81, 84, 82, 3, 69,

In [ ]:
# tar에 있는 '\t' 토큰 제거 -> Dense 와 softmax 함수 위에서 단어를 출력하기 때문 SOS 토큰은 필요없음
decoder_target = []
for data in df.tar:
    timestep = 0
    encoded_line = []
    for char in data:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep += 1
    decoder_target.append(encoded_line)
print('tar encoding: ', decoder_target[:5])

tar encoding:  [[3, 35, 72, 65, 75, 68, 3, 36, 78, 75, 78, 81, 72, 77, 70, 9, 3, 72, 82, 3, 64, 3, 66, 78, 75, 78, 81, 72, 77, 70, 3, 64, 79, 79, 75, 72, 66, 64, 83, 72, 78, 77, 3, 83, 71, 64, 83, 3, 64, 75, 75, 78, 86, 82, 3, 88, 78, 84, 3, 83, 78, 3, 68, 87, 79, 68, 81, 72, 68, 77, 66, 68, 3, 65, 68, 64, 84, 83, 72, 69, 84, 75, 3, 82, 83, 78, 81, 72, 68, 82, 3, 72, 77, 3, 83, 71, 68, 3, 35, 72, 65, 75, 68, 16, 3, 2], [3, 37, 78, 3, 88, 78, 84, 3, 86, 78, 81, 74, 3, 64, 83, 3, 64, 3, 36, 72, 83, 88, 3, 65, 64, 77, 74, 33, 3, 2], [3, 49, 54, 51, 42, 53, 48, 9, 82, 3, 65, 68, 82, 83, 82, 68, 75, 75, 68, 81, 14, 3, 86, 71, 72, 66, 71, 3, 81, 68, 66, 78, 81, 67, 68, 67, 3, 22, 83, 71, 3, 81, 78, 84, 70, 71, 3, 15, 66, 84, 83, 82, 3, 65, 88, 3, 86, 78, 81, 67, 82, 3, 78, 69, 3, 76, 78, 84, 83, 71, 3, 69, 81, 78, 76, 3, 64, 65, 81, 78, 64, 67, 16, 3, 2], [3, 42, 77, 3, 36, 71, 64, 79, 83, 68, 81, 3, 19, 19, 3, 43, 68, 82, 84, 82, 3, 66, 64, 75, 75, 68, 67, 3, 45, 64, 89, 64, 81, 84, 82, 3, 

In [ ]:
max_src_len = max([len(data) for data in df.src])
max_tar_len = max([len(data) for data in df.tar])
print(max_src_len)
print(max_tar_len)

86
222


In [ ]:
# 패딩 한국어는 최대 길이 30, 영어는 70
encoder_input = pad_sequences(encoder_input, maxlen=30, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=70 ,padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=70, padding='post')

In [ ]:
# 원-핫 인코딩
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# Seq2Seq

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

### Encoder

In [ ]:
encoder_input.shape, decoder_input.shape, decoder_target.shape

((10000, 30, 1391), (10000, 70, 116), (10000, 70, 116))

In [ ]:
encoder_inputs = Input(shape=(None, encoder_input.shape[2]))
encoder_lstm = LSTM(units=256, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# 은닉셀과, 셀상태
encoder_states = [state_h, state_c]

### Decoder

In [ ]:
decoder_inputs = Input(shape=(None, decoder_input.shape[2]))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_sotfmax_layer = Dense(decoder_input.shape[2], activation='softmax')
decoder_outputs = decoder_sotfmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy')

In [ ]:
model.fit(
    x=[encoder_input, decoder_input], 
    y=decoder_target, 
    batch_size=64, 
    epochs=40, 
    validation_split=0.2,
)

Epoch 1/40


2022-12-10 00:56:59.546731: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


125/125 [==============================] - 29s 224ms/step - loss: 2.8224 - val_loss: 2.2181
Epoch 2/40
125/125 [==============================] - 27s 216ms/step - loss: 1.9452 - val_loss: 1.7668
Epoch 3/40
125/125 [==============================] - 28s 228ms/step - loss: 1.5287 - val_loss: 1.4710
Epoch 4/40
125/125 [==============================] - 27s 217ms/step - loss: 1.2429 - val_loss: 1.2748
Epoch 5/40
125/125 [==============================] - 26s 212ms/step - loss: 1.0831 - val_loss: 2.0396
Epoch 6/40
125/125 [==============================] - 27s 220ms/step - loss: 1.0277 - val_loss: 1.0180
Epoch 7/40
125/125 [==============================] - 28s 224ms/step - loss: 0.8479 - val_loss: 0.9738
Epoch 8/40
125/125 [==============================] - 28s 227ms/step - loss: 0.8109 - val_loss: 0.9284
Epoch 9/40
125/125 [==============================] - 28s 224ms/step - loss: 0.7742 - val_loss: 0.9103
Epoch 10/40
125/125 [==============================] - 28s 224ms/step - loss: 0.7490

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 1391)  0           []                               
                                ]                                                                 
                                                                                                  
 input_2 (InputLayer)           [(None, None, 116)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        1687552     ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                 

## 번역기 동작 시키기

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어 예측을 위해서 초기상태 를 이전 시점의 상태로 사용
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉상태와 셀상태를 버리지 않음
decoder_states = [state_h, state_c]
decoder_outputs = decoder_sotfmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_state_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict([input_seq])
    
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, decoder_input.shape[2]))
    target_seq[0, 0, tar_to_index['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ""
    
    # stop condition이 True될 때까지 루프 반복
    while not stop_condition:
        # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
        
        # <EOS>에 도달하거나 최대 길이를 넘으면 중단
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True
        
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, decoder_input.shape[2]))
        target_seq[0, 0, sampled_token_index] = 1.
        
        #현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value= [h, c]
        
    return decoded_sentence


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from statistics import median

In [ ]:
def translate_seq2seq(show_text=None):
    bleu_score = []
    for seq_index in [1, 20, 30, 40, 50, 60, 100, 200, 300, 400, 500, 1000]: # 입력 문장 인덱스
        input_seq = encoder_input[seq_index:seq_index+1]
        decoded_sentence = decode_sequence(input_seq)
        score = sentence_bleu(
            list(map(lambda ref: ref.split(), df.tar[seq_index][2:len(df.tar[seq_index])-1])),
            df.tar[seq_index][2:len(df.tar[seq_index])-1].split(),
            weights=(1, 0, 0, 0)
        )
        
        bleu_score.append(score)
        if show_text:
            print(35 * '-')
            print('입력문장: ', df.src[seq_index])
            print('정답문장: ', df.tar[seq_index][2:len(df.tar[seq_index])-1]) # \t 와 \n 빼고 출력
            print('번역문장: ', decoded_sentence[1:len(decoded_sentence)-1]) # \n을 빼고 출력
        
    return print("1gram bleu Score:{0:0.4f}".format(median(bleu_score) * 100))

-----------------------------------
입력문장:  씨티은행에서 일하세요?
정답문장:  Do you work at a City bank? 
번역문장:  Could you let me know the March Mark? 
-----------------------------------
입력문장:  이 제품을 사용할 때 예기치 않은 추락의 경우를 고려하면, 안전을 생각하여 잔디밭이 필요합니다.
정답문장:  Also, this product needs a grass field for safety considering unexpected falls. 
번역문장:  I will send you the divers after the price after 20 years ago. 
-----------------------------------
입력문장:  추가로 작성해서 내일까지 보내줄게요.
정답문장:  I'll fill it out and send it to you by tomorrow. 
번역문장:  I will send you the best after company and the send of the stope. 
-----------------------------------
입력문장:  "유괴"는 스코틀랜드를 배경으로 한 모험 소설이며 문학적 완성도 또한 높은 소설이에요.
정답문장:  "Kidnapped" is an adventure story set in Scotland with high literary significance. 
번역문장:  And the send of the second floor selected and sent is below. 
-----------------------------------
입력문장:  "공식 초청 레터"를 받는 데로 비자를 받을 것입니다.
정답문장:  I will get my visa as soon as I receive the " official invitation letter". 
번역